# Qiskit VQE Tutorial
Full tutorial using Qiskit VQE class following
https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/02_vqe_advanced_options.ipynb

In [1]:
import numpy as np
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import NELDER_MEAD
from qiskit_nature.circuit.library.ansatzes import UCC
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit.utils import algorithm_globals
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
import datetime

from qiskit import QuantumCircuit

from src.hamiltonian.FermionicHamiltonian import FermionicHamiltonian
from src.molecule.BeH2 import BeH2

In [17]:
# Hamiltonian operator creation
H2_op = SparsePauliOp.from_list(
    [
        ("IIII", -1.052373245772859),
        ("IZII", 0.39793742484318045),
        ("ZIII", -0.39793742484318045),
        ("ZZII", -0.01128010425623538),
        ("XXII", 0.18093119978423156),
    ]
)

In [18]:
H2_op.num_qubits

In [2]:
molecule = BeH2("BeH2", 14, 6)

In [3]:
hamiltonian = FermionicHamiltonian(molecule.num_orbitals, molecule=molecule)
op = hamiltonian.get_hamiltonian()

In [4]:
print(len(op))

24598


In [5]:
# create estimator, optimizer, converter, ansatz
estimator = Estimator()
# optimizer definition
optimizer = NELDER_MEAD(maxiter=10)
# use UCC Ansatz
converter = QubitConverter(mapper=JordanWignerMapper(), two_qubit_reduction=True)
#ansatz = UCC(excitations='st')

In [6]:
ucc_ansatz = UCC(qubit_converter=converter, num_spin_orbitals=14, num_particles=(0,6), excitations='sd', alpha_spin=True, beta_spin=True, max_spin_excitation=1, generalized=True, preserve_spin=True, reps=5)

In [7]:
ucc_ansatz.num_qubits
# has to be equal to the ones of the hamiltonian

14

In [11]:
ucc_ansatz_h2 = UCC(qubit_converter=converter, num_spin_orbitals=4, num_particles=(0,2), excitations='sd', alpha_spin=True, beta_spin=True, max_spin_excitation=1, generalized=True, preserve_spin=True, reps=5)

In [8]:
print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
algorithm_globals.random_seed = 50

# use ansatz TwoLocal
#ansatz2 = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
counts = []
values = []

Optimizer: NELDER_MEAD        

In [9]:
def store_intermediate_result(eval_count, parameters, mean, std):
    time = datetime.datetime.now()
    print(f"Iteration done at {time}")
    counts.append(eval_count)
    values.append(mean)

In [12]:
# TwoLocal Ansatz
# ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
optimizer.setting

"Optimizer: NELDER_MEAD\n-- method: nelder-mead\n-- bounds_support_level: 1\n-- gradient_support_level: 1\n-- initial_point_support_level: 3\n-- options: {'maxiter': 10, 'maxfev': 1000, 'disp': False, 'xatol': 0.0001, 'adaptive': False}\n-- max_evals_grouped: 1\n-- kwargs: {'tol': None}\n"

In [ ]:
# Use VQE with estimator, ansatz, optimizer
vqe = VQE(estimator, ucc_ansatz, optimizer, callback=store_intermediate_result)
# invoke computation on the operator
time = datetime.datetime.now()
print(f"Starting calculation at: {time}")
result = vqe.compute_minimum_eigenvalue(operator=op)
time = datetime.datetime.now()
print(f"Ended calculation at: {time}")


converge_counts = np.asarray(counts)
converge_vals = np.asarray(values)

print("\rOptimization complete");

Starting calculation at: 2023-04-25 13:39:56.104263
Iteration done at 2023-04-25 13:43:18.047229
Iteration done at 2023-04-25 13:46:35.037195
Iteration done at 2023-04-25 13:50:03.960707
Iteration done at 2023-04-25 13:53:23.450526
Iteration done at 2023-04-25 13:56:37.601574
Iteration done at 2023-04-25 13:59:52.681036
Iteration done at 2023-04-25 14:02:34.213550
Iteration done at 2023-04-25 14:05:48.731591
Iteration done at 2023-04-25 14:09:01.468149
Iteration done at 2023-04-25 14:12:27.541346
Iteration done at 2023-04-25 14:15:52.679479
Iteration done at 2023-04-25 14:19:05.941763
Iteration done at 2023-04-25 14:22:14.844305
Iteration done at 2023-04-25 14:25:50.780807
Iteration done at 2023-04-25 14:28:57.192836
Iteration done at 2023-04-25 14:31:58.540455
Iteration done at 2023-04-25 14:35:14.375787
Iteration done at 2023-04-25 14:38:35.872309


In [ ]:
# Outputting the optimization process
import pylab

pylab.rcParams["figure.figsize"] = (12, 8)
pylab.plot(converge_counts, converge_vals, label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers")
pylab.legend(loc="upper right");

In [ ]:
# Analytic solution
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import PauliSumOp

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=PauliSumOp(H2_op))
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")